# Server-Dispatcher Coupled

Vamos a probar como se comportan el server y el dispatcher aisladamente. Crearemos un dispatcher y 3 servers libres.

In [35]:
from pringles.simulator import Simulator
mySimulator = Simulator(cdpp_bin_path='bin/', user_models_dir='src/')

atomics = dict([(atomic.__name__, atomic) for atomic in mySimulator.atomic_registry.discovered_atomics])
Server = atomics['Server']
Dispatcher = atomics['Dispatcher']

Los servers se inicializarán en el estado dado, y el dispatcher deberá conocer dicha configuración inicial también.

In [36]:
from pringles.models import Coupled
number_of_servers = 3
a_dispatcher = Dispatcher("a_dispatcher", 
                          numberOfServers=number_of_servers, 
                          server0="free",
                          server1="free",
                          server2="free")

servers = [Server("server" + str(i), initialStatus="free", mean=10.0, setupTime="0:0:10:0") for i in range(number_of_servers)]
    
print("--Dispatcher ports")
print("Inport names: ", [port.name for port in a_dispatcher.inports])
print("Outport names: ", [port.name for port in a_dispatcher.outports])

print("--Server ports--")
print("Inport names: ", [port.name for port in servers[0].inports])
print("Outport names: ", [port.name for port in servers[0].outports])

--Dispatcher ports
Inport names:  ['newJob', 'jobDone', 'serverStatus']
Outport names:  ['requestJob', 'server0', 'server1', 'server2']
--Server ports--
Inport names:  ['job', 'powerSignal']
Outport names:  ['done', 'ready']


Ahora conectaremos las componentes:
- El dispatcher se conectará con un puerto dedicado para cada server para pedirle que procese un job.
- Los servers avisarán al dispatcher a través de su puerto de salida 'jobDone' cuando terminen el job.
- A través del puerto de salida 'requestJob' del acoplado Top, se simulará cuando el dispatcher tiene servidores libres y pide un job a la Queue para procesar un nuevo job.
- A través del puerto de entrada 'newJob' del acoplado Top, se simulará cuando la Queue le envía al dispatcher un nuevo job.
- A través del puerto de entrada 'serverStatus' del acoplado Top, se simulará cuando el AutoScaler le avisa al dispatcher sobre nuevos estados de los servidores.

In [37]:
subcomponents = [a_dispatcher] + servers
    
top_model = (Coupled(name='top', subcomponents=subcomponents)
             .add_inport("newJob")
             .add_inport("jobDone")             
             .add_inport("serverStatus")
             .add_outport("requestJob")
             
             .add_coupling('newJob', a_dispatcher.get_port("newJob"))
             .add_coupling('jobDone', a_dispatcher.get_port("jobDone"))
             .add_coupling('serverStatus', a_dispatcher.get_port("serverStatus"))
             
             .add_coupling(a_dispatcher.get_port('requestJob'), "requestJob")
             .add_coupling(a_dispatcher.get_port('server0'), servers[0].get_port('job'))
             .add_coupling(a_dispatcher.get_port('server1'), servers[1].get_port('job'))
             .add_coupling(a_dispatcher.get_port('server2'), servers[2].get_port('job'))
             
             .add_coupling(servers[0].get_port('done'), a_dispatcher.get_port('jobDone'))
             .add_coupling(servers[1].get_port('done'), a_dispatcher.get_port('jobDone'))
             .add_coupling(servers[2].get_port('done'), a_dispatcher.get_port('jobDone'))
             
            )
top_model

In [38]:
from pringles.simulator import Simulation, Event
from pringles.utils import VirtualTime

sim_events = [
    # Empiezan los 3 servidores prendidos. Y se avisa que el 2 está apagado (habría que mandarle tmb que se apague)
    Event(VirtualTime(0,0,20,0,0), top_model.get_port('serverStatus'), [float(2),float(0)]),
    # Llega un Job a los 25 y el dispatcher lo debería mandar al server 0 porque es el de menor ID
    Event(VirtualTime(0,0,25,0,0), top_model.get_port('newJob'), float(1)),
    # Llega que se prendió de nuevo el server 2
    Event(VirtualTime(0,0,35,0,0), top_model.get_port('serverStatus'), [float(2),float(1)]),
    # Llega otro Job a los 38 y se los manda de nuevo al server 0 (en la simulación el server 0 termina el job anterior
    # en el segundo 36)
    Event(VirtualTime(0,0,38,0,0), top_model.get_port('newJob'), float(1)),
    # Llega en 40 que se apagó el 0, pero el lo tenía en busy, espera que se apague y lo pasa a off
    Event(VirtualTime(0,0,40,0,0), top_model.get_port('serverStatus'), [float(0),float(0)]),
    # Llega en 42 un Job y como el 0 está en apagado/busy, se lo manda al server 1
    Event(VirtualTime(0,0,42,0,0), top_model.get_port('newJob'), float(1)),
    # Llega en 43 que se apague el 1, espera que se termine el job y lo apaga
    Event(VirtualTime(0,0,43,0,0), top_model.get_port('serverStatus'), [float(1),float(0)]),
    # Llega en 44 un Job y se lo manda al server 2, el único prendido
    Event(VirtualTime(0,0,44,0,0), top_model.get_port('newJob'), float(1)),
]

a_simulation = Simulation(top_model = top_model, 
                          duration = VirtualTime.of_minutes(50), 
                          events=sim_events,
                          working_dir='sim_results/server-dispatcher'
                         )

results = mySimulator.run_simulation(a_simulation)

In [39]:
print(results.get_process_output())

PCD++: A Tool to Implement n-Dimensional Cell-DEVS models
Version 3.0 - March 2003
Troccoli A., Rodriguez D., Wainer G., Barylko A., Beyoglonian J., Lopez A.
-----------------------------------------------------------------------------
PCD++ Extended States: An extended and improved version of CD++ for Cell-DEVS
Version 4.1.2 - December 2018
Santi L., Castro, R., Pimás, J.
-----------------------------------------------------------------------------
Discrete Event Simulation Lab
Departamento de Computación
Facultad de Ciencias Exactas y Naturales
Universidad de Buenos Aires, Argentina
-----------------------------------------------------------------------------
Compiled for standalone simulation


Loading models from sim_results/server-dispatcher/2019-11-16-191630-a067c772d34149ed9c4d83167ce7e299/top_model
Loading events from sim_results/server-dispatcher/2019-11-16-191630-a067c772d34149ed9c4d83167ce7e299/events
Running parallel simulation. Reading models partition from 
Model partitio

In [40]:
# En los evetos podemos ver que llegan 4 jobs
# Por lo que debemos ver 5 requests de jobs pues todos los jobs son terminados en la simulación.
display(results.output_df)

,time,port,value
0,00:00:00:000,requestjob,1.0
1,00:00:25:000,requestjob,1.0
2,00:00:38:000,requestjob,1.0


Como podemos observar, los mensajes que siguen abajo, son los mismos que en el notebook sin la Queue.

In [41]:
# Veamos que le llegan al server 0 los primeros 2 jobs, hasta que lo apagan en segundo 40
display(results.logs_dfs['server0'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:25:000,top(05),job,0.0,server0(02)


In [42]:
# Veamos que el siguiente job le llega al server 1 en el segundo 42, y luego lo apagan en el segundo 43
# Y se espera a que termine para apagarlo
display(results.logs_dfs['server1'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:38:000,top(05),job,1.0,server1(03)


In [43]:
# Por último cuando el 0 y el 1 están apagados, cuando llega el último job en el segundo 44, se lo envía al server 2
# que lo termina (ver logs dispatcher)
display(results.logs_dfs['server2'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:42:000,top(05),job,2.0,server2(04)


In [44]:
display(results.logs_dfs['a_dispatcher'])

,0,1,message_type,time,model_origin,port,value,model_dest
0,0,L,X,00:00:20:000,top(05),serverstatus,"(2.0, 0.0)",a_dispatcher(01)
1,0,L,X,00:00:25:000,top(05),newjob,1,a_dispatcher(01)
2,0,L,X,00:00:35:000,top(05),serverstatus,"(2.0, 1.0)",a_dispatcher(01)
3,0,L,X,00:00:38:000,top(05),newjob,1,a_dispatcher(01)
4,0,L,X,00:00:40:000,top(05),serverstatus,"(0.0, 0.0)",a_dispatcher(01)
5,0,L,X,00:00:42:000,top(05),newjob,1,a_dispatcher(01)
6,0,L,X,00:00:43:000,top(05),serverstatus,"(1.0, 0.0)",a_dispatcher(01)
7,0,L,X,00:00:44:000,top(05),newjob,1,a_dispatcher(01)
